In [1]:
import pandas as pd
import os
os.chdir('..')

In [2]:
df_android_raw = df = pd.read_csv("data/android_sdk_totins_31oct2023.csv", engine="python")
df_ios_raw = pd.read_csv('data/ios_sdk_totins_31oct2023.csv')

In [3]:
df_android_raw.head()

,id,android_totins_31oct2023,name,company,url,_function,other_stores
0,10,79418,AdColony,None,adcolony.com,AD_NETWORK,"[{'store': 'itunes_connect', 'id': 721}]"
1,100,14,Aurasma,None,Aurasma.com,MEDIA_PLAYER,[]
2,1000,199650,Android Pay,None,https://developers.google.com/android/,DEV_TOOL,[]
3,1001,849357,Firebase,None,https://firebase.google.com/,ANALYTICS,"[{'store': 'itunes_connect', 'id': 4611}]"
4,1002,1059,GetUI,None,http://www.getui.com/,PUSH_MESSAGING,[]


In [4]:
df_ios_raw.head()

,id,ios_totins_31oct2023,name,company,url,_function,other_stores
0,1004,245,UISS,None,None,DEV_TOOL,[]
1,104,880,RFRateMe,None,https://www.cocoacontrols.com/controls/rfrateme,SUPPORT_REVIEWS,[]
2,105,7455,RevMob,None,revmob.com,AD_NETWORK,"[{'store': 'google_play', 'id': 64}]"
3,1054,844,Typhoon,None,None,DEV_TOOL,[]
4,1061,11565,TwinPush,None,twinpush.com,PUSH_MESSAGING,[]


In [5]:
df_android_raw = df_android_raw.dropna(subset=["name"])
df_ios_raw = df_ios_raw.dropna(subset=["name"])

In [6]:
df_ios_raw[df_ios_raw['name'] == 'GetUI']

,id,ios_totins_31oct2023,name,company,url,_function,other_stores
401,4574,9284,GetUI,None,http://www.getui.com/,PUSH_MESSAGING,[]


In [7]:
df_android = df_android_raw.copy()
df_ios = df_ios_raw.copy()

In [8]:
import ast

def extract_store_id(x):
    try:
        data = ast.literal_eval(x) if isinstance(x, str) else x
        if data and isinstance(data, list):
            return data[0].get("id")
    except Exception:
        return None
    return None

df_android["ios_id"] = df_android["other_stores"].apply(extract_store_id)
df_ios["android_id"] = df_ios["other_stores"].apply(extract_store_id)

In [9]:
df_android["ios_id"] = df_android["ios_id"].astype("Int64")
df_ios["android_id"] = df_ios["android_id"].astype("Int64")

In [10]:
df_android = df_android.drop(columns=["other_stores"])
df_ios = df_ios.drop(columns=["other_stores"])

In [11]:
print(df_android.shape)
print(df_ios.shape)

(1614, 7)
(1362, 7)


In [12]:
df_android.head()

,id,android_totins_31oct2023,name,company,url,_function,ios_id
0,10,79418,AdColony,None,adcolony.com,AD_NETWORK,721
1,100,14,Aurasma,None,Aurasma.com,MEDIA_PLAYER,<NA>
2,1000,199650,Android Pay,None,https://developers.google.com/android/,DEV_TOOL,<NA>
3,1001,849357,Firebase,None,https://firebase.google.com/,ANALYTICS,4611
4,1002,1059,GetUI,None,http://www.getui.com/,PUSH_MESSAGING,<NA>


In [13]:
df_android["ios_id"].isna().sum()

641

In [14]:
df_ios.head()

,id,ios_totins_31oct2023,name,company,url,_function,android_id
0,1004,245,UISS,None,None,DEV_TOOL,<NA>
1,104,880,RFRateMe,None,https://www.cocoacontrols.com/controls/rfrateme,SUPPORT_REVIEWS,<NA>
2,105,7455,RevMob,None,revmob.com,AD_NETWORK,64
3,1054,844,Typhoon,None,None,DEV_TOOL,<NA>
4,1061,11565,TwinPush,None,twinpush.com,PUSH_MESSAGING,<NA>


In [15]:
df_ios["android_id"].isna().sum()

381

In [16]:
df_combined = pd.merge(
    df_android,
    df_ios,
    on="name",
    how="outer",
    suffixes=("_orig_android", "_orig_ios")
)
df_combined

,id_orig_android,android_totins_31oct2023,name,company_orig_android,url_orig_android,_function_orig_android,ios_id,id_orig_ios,ios_totins_31oct2023,company_orig_ios,url_orig_ios,_function_orig_ios,android_id
0,10.0,79418.0,AdColony,None,adcolony.com,AD_NETWORK,721,721.0,39734.0,None,adcolony.com,AD_NETWORK,10
1,100.0,14.0,Aurasma,None,Aurasma.com,MEDIA_PLAYER,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>
2,1000.0,199650.0,Android Pay,None,https://developers.google.com/android/,DEV_TOOL,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>
3,1001.0,849357.0,Firebase,None,https://firebase.google.com/,ANALYTICS,4611,4611.0,245060.0,None,https://firebase.google.com,ANALYTICS,1001
4,1002.0,1059.0,GetUI,None,http://www.getui.com/,PUSH_MESSAGING,<NA>,4574.0,9284.0,None,http://www.getui.com/,PUSH_MESSAGING,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,NaN,NaN,Woodo,NaN,NaN,NaN,<NA>,845.0,3.0,None,woodo.tv,VIDEO_AD,<NA>
2011,NaN,NaN,Weixin,NaN,NaN,NaN,<NA>,862.0,3507.0,None,None,DEV_PLATFORM,<NA>
2012,NaN,NaN,WebViewJavascriptBridge,NaN,NaN,NaN,<NA>,868.0,12265.0,None,https://github.com/marcuswestin/WebViewJavascr...,DEV_TOOL,<NA>
2013,NaN,NaN,Vertigo,NaN,NaN,NaN,<NA>,947.0,457.0,None,vertigo.com,DEV_TOOL,<NA>


In [17]:
# --- Platforms column
def get_platforms(row):
    platforms = []
    if pd.notna(row.get("id_orig_android")):
        platforms.append("android")
    if pd.notna(row.get("id_orig_ios")):
        platforms.append("ios")
    return platforms

df_combined["platforms"] = df_combined.apply(get_platforms, axis=1)

# --- totins column
df_combined["totins"] = (
    df_combined["android_totins_31oct2023"].fillna(0)
    + df_combined["ios_totins_31oct2023"].fillna(0)
)

# --- Reconcile company, url, and _function
def reconcile(a, b):
    if pd.isna(a) and pd.isna(b):
        return []
    if a == b:
        return [a] if pd.notna(a) else [b]
    return [x for x in [a, b] if pd.notna(x)]

for col in ["company", "url", "_function"]:
    df_combined[col] = df_combined.apply(
        lambda row: reconcile(row.get(f"{col}_orig_android"), row.get(f"{col}_orig_ios")), axis=1
    )

In [18]:
df_combined

,id_orig_android,android_totins_31oct2023,name,company_orig_android,url_orig_android,_function_orig_android,ios_id,id_orig_ios,ios_totins_31oct2023,company_orig_ios,url_orig_ios,_function_orig_ios,android_id,platforms,totins,company,url,_function
0,10.0,79418.0,AdColony,None,adcolony.com,AD_NETWORK,721,721.0,39734.0,None,adcolony.com,AD_NETWORK,10,"[android, ios]",119152.0,[None],[adcolony.com],[AD_NETWORK]
1,100.0,14.0,Aurasma,None,Aurasma.com,MEDIA_PLAYER,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,[android],14.0,[None],[Aurasma.com],[MEDIA_PLAYER]
2,1000.0,199650.0,Android Pay,None,https://developers.google.com/android/,DEV_TOOL,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,[android],199650.0,[None],[https://developers.google.com/android/],[DEV_TOOL]
3,1001.0,849357.0,Firebase,None,https://firebase.google.com/,ANALYTICS,4611,4611.0,245060.0,None,https://firebase.google.com,ANALYTICS,1001,"[android, ios]",1094417.0,[None],"[https://firebase.google.com/, https://firebas...",[ANALYTICS]
4,1002.0,1059.0,GetUI,None,http://www.getui.com/,PUSH_MESSAGING,<NA>,4574.0,9284.0,None,http://www.getui.com/,PUSH_MESSAGING,<NA>,"[android, ios]",10343.0,[None],[http://www.getui.com/],[PUSH_MESSAGING]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,NaN,NaN,Woodo,NaN,NaN,NaN,<NA>,845.0,3.0,None,woodo.tv,VIDEO_AD,<NA>,[ios],3.0,[None],[woodo.tv],[VIDEO_AD]
2011,NaN,NaN,Weixin,NaN,NaN,NaN,<NA>,862.0,3507.0,None,None,DEV_PLATFORM,<NA>,[ios],3507.0,[None],[None],[DEV_PLATFORM]
2012,NaN,NaN,WebViewJavascriptBridge,NaN,NaN,NaN,<NA>,868.0,12265.0,None,https://github.com/marcuswestin/WebViewJavascr...,DEV_TOOL,<NA>,[ios],12265.0,[None],[https://github.com/marcuswestin/WebViewJavasc...,[DEV_TOOL]
2013,NaN,NaN,Vertigo,NaN,NaN,NaN,<NA>,947.0,457.0,None,vertigo.com,DEV_TOOL,<NA>,[ios],457.0,[None],[vertigo.com],[DEV_TOOL]


In [19]:
df_combined[df_combined['name'] == 'GetUI']

,id_orig_android,android_totins_31oct2023,name,company_orig_android,url_orig_android,_function_orig_android,ios_id,id_orig_ios,ios_totins_31oct2023,company_orig_ios,url_orig_ios,_function_orig_ios,android_id,platforms,totins,company,url,_function
4,1002.0,1059.0,GetUI,None,http://www.getui.com/,PUSH_MESSAGING,<NA>,4574.0,9284.0,None,http://www.getui.com/,PUSH_MESSAGING,<NA>,"[android, ios]",10343.0,[None],[http://www.getui.com/],[PUSH_MESSAGING]


In [20]:
df_combined = df_combined.rename(columns={
    "ios_id": "ios_id_from_android_perspective",
    "android_id": "android_id_from_ios_perspective",
    "_function": "function"
})

In [21]:
df_combined = df_combined.drop(
    columns=["company_orig_android", "company_orig_ios", 
             "url_orig_android", "url_orig_ios", 
             "_function_orig_android", "_function_orig_ios"]
)
df_combined.head()

,id_orig_android,android_totins_31oct2023,name,ios_id_from_android_perspective,id_orig_ios,ios_totins_31oct2023,android_id_from_ios_perspective,platforms,totins,company,url,function
0,10.0,79418.0,AdColony,721,721.0,39734.0,10,"[android, ios]",119152.0,[None],[adcolony.com],[AD_NETWORK]
1,100.0,14.0,Aurasma,<NA>,NaN,NaN,<NA>,[android],14.0,[None],[Aurasma.com],[MEDIA_PLAYER]
2,1000.0,199650.0,Android Pay,<NA>,NaN,NaN,<NA>,[android],199650.0,[None],[https://developers.google.com/android/],[DEV_TOOL]
3,1001.0,849357.0,Firebase,4611,4611.0,245060.0,1001,"[android, ios]",1094417.0,[None],"[https://firebase.google.com/, https://firebas...",[ANALYTICS]
4,1002.0,1059.0,GetUI,<NA>,4574.0,9284.0,<NA>,"[android, ios]",10343.0,[None],[http://www.getui.com/],[PUSH_MESSAGING]


In [22]:
df_combined.rename(columns={
    "id_orig_android": "android_id",
    "id_orig_ios": "ios_id",
    "android_totins_31oct2023": "android_totins",
    "ios_totins_31oct2023": "ios_totins",
    "_function": "function"
}, inplace=True)

df_combined["ios_id"] = df_combined["ios_id"].astype("Int64")
df_combined["android_id"] = df_combined["android_id"].astype("Int64")
df_combined["android_totins"] = df_combined["android_totins"].astype("Int64")
df_combined["ios_totins"] = df_combined["ios_totins"].astype("Int64")
df_combined["totins"] = df_combined["totins"].astype("Int64")
df_combined["ios_id_from_android_perspective"] = df_combined["ios_id_from_android_perspective"].astype("Int64")
df_combined["android_id_from_ios_perspective"] = df_combined["android_id_from_ios_perspective"].astype("Int64")

In [23]:
df_final = df_combined[[
    "android_id",
    "android_id_from_ios_perspective",
    "ios_id",
    "ios_id_from_android_perspective",
    "name",
    "company",
    "android_totins",
    "ios_totins",
    "totins",
    "function",
    "platforms",
    "url"
]].copy()

In [24]:
df_final.head()

,android_id,android_id_from_ios_perspective,ios_id,ios_id_from_android_perspective,name,company,android_totins,ios_totins,totins,function,platforms,url
0,10,10,721,721,AdColony,[None],79418,39734,119152,[AD_NETWORK],"[android, ios]",[adcolony.com]
1,100,<NA>,<NA>,<NA>,Aurasma,[None],14,<NA>,14,[MEDIA_PLAYER],[android],[Aurasma.com]
2,1000,<NA>,<NA>,<NA>,Android Pay,[None],199650,<NA>,199650,[DEV_TOOL],[android],[https://developers.google.com/android/]
3,1001,1001,4611,4611,Firebase,[None],849357,245060,1094417,[ANALYTICS],"[android, ios]","[https://firebase.google.com/, https://firebas..."
4,1002,<NA>,4574,<NA>,GetUI,[None],1059,9284,10343,[PUSH_MESSAGING],"[android, ios]",[http://www.getui.com/]


In [25]:
df_final.shape

(2015, 12)

In [26]:
# Pick first element of any column that has value of list.

df_final["function"] = df_final["function"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_final["url"] = df_final["url"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_final["company"] = df_final["company"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)

df_final["company"] = df_final["company"].apply(
    lambda x: (
        [pd.NA if (v is None or str(v).lower() == "none") else v for v in x]
        if isinstance(x, list) 
        else (pd.NA if (x is None or str(x).lower() == "none") else x)
    )
)
df_final["url"] = df_final["url"].apply(
    lambda x: (
        [pd.NA if (v is None or str(v).lower() == "none") else v for v in x]
        if isinstance(x, list) 
        else (pd.NA if (x is None or str(x).lower() == "none") else x)
    )
)

In [27]:
df_final.head()

,android_id,android_id_from_ios_perspective,ios_id,ios_id_from_android_perspective,name,company,android_totins,ios_totins,totins,function,platforms,url
0,10,10,721,721,AdColony,<NA>,79418,39734,119152,AD_NETWORK,"[android, ios]",adcolony.com
1,100,<NA>,<NA>,<NA>,Aurasma,<NA>,14,<NA>,14,MEDIA_PLAYER,[android],Aurasma.com
2,1000,<NA>,<NA>,<NA>,Android Pay,<NA>,199650,<NA>,199650,DEV_TOOL,[android],https://developers.google.com/android/
3,1001,1001,4611,4611,Firebase,<NA>,849357,245060,1094417,ANALYTICS,"[android, ios]",https://firebase.google.com/
4,1002,<NA>,4574,<NA>,GetUI,<NA>,1059,9284,10343,PUSH_MESSAGING,"[android, ios]",http://www.getui.com/


In [28]:
df_final = df_final.sort_values(by="totins", ascending=False).reset_index(drop=True)
df_final.head()

,android_id,android_id_from_ios_perspective,ios_id,ios_id_from_android_perspective,name,company,android_totins,ios_totins,totins,function,platforms,url
0,962,962,4500,4500,Google Mobile Ads (AdMob),<NA>,1018573,282919,1301492,AD_NETWORK,"[android, ios]",<NA>
1,566,566,350,350,Google,<NA>,1222566,54686,1277252,DEV_PLATFORM,"[android, ios]",google.com
2,1001,1001,4611,4611,Firebase,<NA>,849357,245060,1094417,ANALYTICS,"[android, ios]",https://firebase.google.com/
3,992,992,5146,5146,Google Sign-In,<NA>,965611,51321,1016932,SOCIAL,"[android, ios]",https://developers.google.com/android/
4,1061,1061,4612,4612,Google Analytics for Firebase,<NA>,699395,222457,921852,ANALYTICS,"[android, ios]",https://firebase.google.com/docs/analytics/


In [29]:
df_final.shape

(2015, 12)

In [30]:
df_final.to_csv("data/sdk_combined.csv", index=False)